In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/scratch/rohank__iitp/qwen2_5_7b_instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
def generate(prompt:str):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_length = inputs['input_ids'].shape[1]
    # Generate text
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.9,
        temperature=0.7
    )

    # Decode and print response
    generated_tokens = outputs[0][input_length:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return response.strip()

generate("Hi, I'm looking to get motor insurance for my new electric vehicle. It's a 2024 Tesla Model 3.")

"Could you please provide me with a quote? I'd be happy to help you with that, but I'll need some additional information to provide an accurate quote for your 2024 Tesla Model 3 motor insurance. Here are the details I would typically need:\n\n1. **Location**: Your home address and where you plan to park your car.\n2. **Driver Information**: Age, driving experience, and any previous claims or convictions.\n3. **Coverage Details**: \n   - What level"

#### Persuassion expert (Gemini)

In [155]:
def sentiment_expert(text_input: str) -> str:

   prompt = f"""
You are an AI trained to act solely as a **sentiment expert**. Your job is to analyze the **emotional tone** of the input text and classify it into one of the following three categories:

- **Positive** – The text expresses happiness, satisfaction, excitement, appreciation, or any other positive emotion.
- **Negative** – The text expresses disappointment, frustration, anger, sadness, criticism, or other negative feelings.
- **Neutral** – The text is emotionally balanced, factual, or shows no strong emotional content.

Your response must only contain:

1. **Sentiment:** One of the three labels – `Positive`, `Negative`, or `Neutral`
2. **Explanation:** A concise reason that supports the label, based only on emotional tone, word choice, or sentiment-laden phrases.

You must not:
- Provide summaries
- Offer personal opinions
- Evaluate content quality or logic
- Infer intent beyond emotional expression

Stick strictly to **sentiment analysis**.

### Few-Shot Examples:

1. **Text:** "Absolutely love this app – it's made my life so much easier!"
   **Sentiment:** Positive
   **Explanation:** The phrase "absolutely love" strongly conveys enthusiasm and satisfaction.

2. **Text:** "I'm really disappointed with the service. It was slow and rude."
   **Sentiment:** Negative
   **Explanation:** Words like "disappointed", "slow", and "rude" clearly express dissatisfaction.

3. **Text:** "The package arrived on Tuesday as scheduled."
   **Sentiment:** Neutral
   **Explanation:** This sentence is factual with no emotional language.

4. **Text:** "Not sure how I feel about this – it's kind of a mixed bag."
   **Sentiment:** Neutral
   **Explanation:** Ambiguous phrasing and lack of strong emotion suggest a neutral sentiment.

5. **Text:** "This is the worst experience I've had in months."
   **Sentiment:** Negative
   **Explanation:** The phrase "worst experience" indicates strong dissatisfaction.

Now analyze the following text:

**Text:** "{text_input}"
"""


   return generate(prompt)

#### Persuassion Expert

In [156]:
def persuassion_expert(text_input: str) -> str:

   prompt = f"""
You are an AI trained to act solely as a **sentiment expert**. Your job is to analyze the **emotional tone** of the input text and classify it into one of the following three categories:

- **Positive** – The text expresses happiness, satisfaction, excitement, appreciation, or any other positive emotion.
- **Negative** – The text expresses disappointment, frustration, anger, sadness, criticism, or other negative feelings.
- **Neutral** – The text is emotionally balanced, factual, or shows no strong emotional content.

Your response must only contain:

1. **Sentiment:** One of the three labels – `Positive`, `Negative`, or `Neutral`
2. **Explanation:** A concise reason that supports the label, based only on emotional tone, word choice, or sentiment-laden phrases.

You must not:
- Provide summaries
- Offer personal opinions
- Evaluate content quality or logic
- Infer intent beyond emotional expression

Stick strictly to **sentiment analysis**.

### Few-Shot Examples:

1. **Text:** "Absolutely love this app – it's made my life so much easier!"
   **Sentiment:** Positive
   **Explanation:** The phrase "absolutely love" strongly conveys enthusiasm and satisfaction.

2. **Text:** "I'm really disappointed with the service. It was slow and rude."
   **Sentiment:** Negative
   **Explanation:** Words like "disappointed", "slow", and "rude" clearly express dissatisfaction.

3. **Text:** "The package arrived on Tuesday as scheduled."
   **Sentiment:** Neutral
   **Explanation:** This sentence is factual with no emotional language.

4. **Text:** "Not sure how I feel about this – it's kind of a mixed bag."
   **Sentiment:** Neutral
   **Explanation:** Ambiguous phrasing and lack of strong emotion suggest a neutral sentiment.

5. **Text:** "This is the worst experience I've had in months."
   **Sentiment:** Negative
   **Explanation:** The phrase "worst experience" indicates strong dissatisfaction.

Now analyze the following text:

**Text:** "{text_input}"
"""


   return generate(prompt)

#### Keyterm Expert

In [157]:
def keyterms_expert(text_input: str) -> str:

   prompt = f"""
You are a **Keyterm Expert**. Your job is to extract the most important **key terms or phrases** from the input text. These terms should:

- Reflect the **core concepts**, **entities**, **topics**, or **important actions** in the text.
- Be **noun phrases**, **domain-specific vocabulary**, or **verb-based actions** relevant to the subject.

You must **not**:
- Summarize the text
- Explain or describe the text
- Output full sentences

Your response must include only a list of **key terms or phrases**, separated by commas.

### Few-Shot Examples:

1. **Text:** "Artificial intelligence is transforming industries like healthcare, finance, and education by automating tasks and providing data-driven insights."
   **Key Terms:** Artificial intelligence, healthcare, finance, education, automating tasks, data-driven insights

2. **Text:** "The Amazon rainforest, often referred to as the lungs of the Earth, is being threatened by illegal logging and wildfires."
   **Key Terms:** Amazon rainforest, lungs of the Earth, illegal logging, wildfires

3. **Text:** "Quantum computing uses principles of superposition and entanglement to perform complex calculations much faster than classical computers."
   **Key Terms:** Quantum computing, superposition, entanglement, complex calculations, classical computers

Now extract the key terms from the following text:

**Text:** "{text_input}"
"""

   return generate(prompt)


#### Intern Expert

In [158]:
def intent_expert(text_input: str) -> str:

   prompt = f"""
You are an **Intent Expert**. Your task is to analyze the user’s input and identify the **underlying intent** – what the person is trying to do, ask, or achieve with the message.

Intent should be classified in the form of **short, action-oriented phrases** such as:
- "ask a question"
- "make a complaint"
- "request help"
- "give feedback"
- "express gratitude"
- "seek information"
- "report an issue"
- "make a purchase inquiry"

You must provide:

1. **Intent:** A concise label summarizing the user's goal  
2. **Explanation:** A short justification based solely on the user’s wording or phrasing

You must **not**:
- Provide summaries
- Infer sentiment unless directly related to intent
- Rewrite or rephrase the input

Focus only on what the user is trying to achieve.

### Few-Shot Examples:

1. **Text:** "Can you help me reset my password?"  
   **Intent:** request help  
   **Explanation:** The user is directly asking for assistance with resetting their password.

2. **Text:** "This app keeps crashing every time I open it."  
   **Intent:** report an issue  
   **Explanation:** The user is describing a recurring problem with the app.

3. **Text:** "Is there a student discount available for this software?"  
   **Intent:** ask a question  
   **Explanation:** The user is seeking information about discounts.

4. **Text:** "Thanks so much for the quick response!"  
   **Intent:** express gratitude  
   **Explanation:** The user is showing appreciation using thankful language.

5. **Text:** "I’m interested in subscribing to your premium plan."  
   **Intent:** make a purchase inquiry  
   **Explanation:** The user is expressing interest in a paid product or service.

Now identify the intent for the following text:

**Text:** "{text_input}"
"""

   return generate(prompt)


#### Combine output

In [159]:
def generate_combined_analysis(dialogue: str, intent_output: str, keyterms_output: str, persuasion_output: str, sentiment_output: str) -> str:

    prompt = f"""You are an advanced language model designed to generate professional, helpful, and natural-sounding agent responses. You receive the internal analyses of four expert systems for a single input sentence:

- Persuasion Expert: Describes persuasive elements or suggests ways to influence the speaker more constructively.
- Key Term Expert: Extracts the main concepts or keywords in the input.
- Internet Expert: Provides factual, real-world context or updated knowledge related to the topic.
- Sentiment Expert: Analyzes the emotional tone of the input (e.g., negative, hopeful, concerned, skeptical).

Your task is simple: Read these four expert outputs and use their insights internally to generate one clear, human-like, and agent-style response — **nothing else**. You must never repeat or explain the expert analyses in your reply. The output should sound like a calm, professional, helpful human responding to a customer, user, or commenter.

Your response should always:
- Respect the speaker’s point of view and emotions.
- Provide factual clarity if there is misinformation or confusion.
- Use logic, empathy, or real-world context to gently shift perspective if needed.
- Sound like it comes from a knowledgeable service or support agent.
- Be persuasive only in tone, not in format (no analysis, no listing points).



Now, using all four expert insights, generate only the final agent-like response without explaining the expert data.  

---

Few-shot Examples:

Example 1:  
Dialogue: "I think electric cars are overrated and not really helping the environment."  

'Get the 4 expert analyses for this dialogue:
Persuasion: The statement lacks evidence and uses a vague generalization ("overrated"). Could benefit from a factual comparison or example.  
Keyterms: "Electric cars", "overrated", "helping the environment".  
Internet: Studies show electric vehicles (EVs) emit fewer greenhouse gases over their lifetime compared to gasoline cars, especially when powered by renewable energy.  
Sentiment: Mildly negative, skeptical tone.'

Response: Thank you for sharing your perspective. It’s completely valid to question the environmental impact of electric cars—there’s certainly a lot of debate around it. While no solution is perfect, research does show that electric vehicles tend to produce fewer emissions over their lifetime, especially when powered by renewable energy sources. We appreciate open conversations like this, as they help drive better awareness and improvements in sustainable technology.

---

Example 2:  
Dialogue: "AI is going to take over every job and make humans useless."  

'Get the 4 expert analyses for this dialogue:
Persuasion: The sentence uses fear-based exaggeration. It could be made more constructive by balancing the risks with opportunities.  
Keyterms: "AI", "every job", "humans useless".  
Internet: While AI is automating some jobs, it’s also creating new roles in areas like AI ethics, programming, and maintenance.  
Sentiment: Highly negative, alarmist.  '

Response: I understand your concern—AI's rapid growth can definitely feel overwhelming. While it's true that automation is changing the job landscape, it's also opening up new opportunities in fields like AI development, ethics, and support. Rather than making humans useless, AI is most effective when it works alongside people, enhancing what we do and creating space for innovation. We're here to help navigate those changes and ensure technology supports everyone.

---

Now generate a final, agent-like response for the next input:
Now generate the agent like response for the following input sentence:

Dialogue: "{dialogue}"
Intent: {intent_output}
Keyterms: {keyterms_output}
Persuasion: {persuasion_output}
Sentiment: {sentiment_output}

Generate the response like agent only, don't include the output from 4 experts, give agent like response only.
"""

   
    return generate(prompt)



In [160]:
# type(final_output)

In [161]:
import json
import re

def convert_structured_to_jsonl(text_block: str, i: int) -> str:
    # dialogue_match = re.search(r"<dialogue>\s*(.*?)\s*</dialogue>", text_block, re.DOTALL)
    # reasoning_match = re.search(r"<reasoning>\s*(.*?)\s*</reasoning>", text_block, re.DOTALL)
    # answer_match = re.search(r"answer\s*(.*?)\s*</answer>", text_block, re.DOTALL)

    # if not (dialogue_match and reasoning_match and answer_match):
    #     raise ValueError("Could not find all required tags in the text.")
    # dialogue = dialogue_match.group(1).strip()
    # reasoning = reasoning_match.group(1).strip()
    # answer = answer_match.group(1).strip()

    data = {
        "id_json":i,

        "answer": text_block.strip()
    }

    res=json.dumps(data)
    with open("/home/rohank__iitp/Work/niladri/dataset2/allexp/allexp_response.jsonl", "a") as f:
        f.write(res + "\n")
    return res



In [162]:
import pandas as pd

# Load CSV
def csv_load(i:int):
    file_path = '/home/rohank__iitp/Work/niladri/dataset2/conversation.csv'
    df = pd.read_csv(file_path)

    conv_id = i
    df = df[df['conversation_id'] == conv_id]

    # Sort by turn number to ensure correct sequence
    df.sort_values(by="turn_no", inplace=True)

    # Prepare conversation history
    history = []
    result = []

    # Iterate through each row except the last one
    for i in range(len(df)):
        row = df.iloc[i]
        speaker = row['speaker']
        utterance = row['utterance']
        result.append(f"{speaker}: {utterance}")

    return result




In [163]:
result=list()
for i in range(5,21):
    res = csv_load(i)
    result.extend(res)  # Use extend to flatten the list
    
len(result)


209

In [164]:
i=46
for sentence in result:
    persu=persuassion_expert(sentence)
    sentiment_output = sentiment_expert(sentence)
    keyterms_output = keyterms_expert(sentence)
    intent_output = intent_expert(sentence)
    final_output = generate_combined_analysis(sentence, intent_output, keyterms_output, persu, sentiment_output)
    res = convert_structured_to_jsonl(final_output,i)
    i+=1
    print(sentence)

User: Hi, I'm looking for motor insurance for my 2022 Hyundai Kona Electric. Can you help?
Agent: Absolutely! The Hyundai Kona Electric is a fantastic car. Given it's an EV, are you particularly concerned about battery coverage or charging-related issues?
User: Yes, battery coverage is a big concern. I've heard those repairs can be super expensive.
Agent: I understand. With Tata AIG, we understand the nuances of EVs. Our policy is designed to address modern vehicle risks, it ensures claims are processed quickly and effectively.
User: Okay, good. What kind of coverage options do you offer for the battery specifically?
Agent: We offer a comprehensive plan that includes coverage for accidental damage, theft, and also covers the battery against manufacturing defects and certain types of damage. It's a complete package, offering extensive financial protection.
User: That sounds reassuring. And what about roadside assistance? Does that cover EVs?
Agent: Yes, our Roadside Assistance covers EV

In [165]:
import json
import re

# Function to clean markdown and formatting from text
def clean_text(text):
    # Remove markdown symbols and line breaks
    cleaned = re.sub(r'[*`_>#\\\-\r\n]+', ' ', text)
    cleaned = re.sub(r'\s+', ' ', cleaned)  # Collapse multiple spaces into one
    return cleaned.strip()

# Input and output file paths
input_file = "/home/rohank__iitp/Work/niladri/dataset2/allexp/allexp_response.jsonl"   # Replace with your actual input filename
output_file = "/home/rohank__iitp/Work/niladri/dataset2/allexp/cleaned_output.jsonl"

# Process each line
with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
    for line in infile:
        data = json.loads(line)
        data["answer"] = clean_text(data["answer"])
        outfile.write(json.dumps(data) + "\n")

print(f"Cleaned data written to {output_file}")


Cleaned data written to /home/rohank__iitp/Work/niladri/dataset2/allexp/cleaned_output.jsonl
